<h1> Tratamento de Dados Faltantes (Missing Values) </h1>
<hr>

<h2> Abordagens mais comuns </h2>

Dados faltantes são um problema bastante comum em aplicações reais. Aplico aqui métodos comuns para contornar esse probelma, tendo em vista que não há solução ideal e geral para todos os casos.

Vou usar os dados de "Preços de Casas", obtidos no kaggle

In [21]:
#!mkdir -p data
#!kaggle competitions download -c house-prices-advanced-regression-techniques -f train.csv -p data


In [22]:
import pandas as pd
import numpy as np

df=pd.read_csv('data/train.csv')

df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


<hr>
<h3>Vou produzir um sumário com a % de dados faltantes em cada coluna;</h3>


A combinuação dos comandos df.isnull().sum() com sort_values ordena os resultados de acordo com o número de dados faltantes

In [23]:
df.isnull().sum().sort_values(ascending=False)[0:20]*100/len(df)
#print(len(df))

PoolQC          99.520548
MiscFeature     96.301370
Alley           93.767123
Fence           80.753425
FireplaceQu     47.260274
LotFrontage     17.739726
GarageYrBlt      5.547945
GarageCond       5.547945
GarageType       5.547945
GarageFinish     5.547945
GarageQual       5.547945
BsmtFinType2     2.602740
BsmtExposure     2.602740
BsmtQual         2.534247
BsmtCond         2.534247
BsmtFinType1     2.534247
MasVnrArea       0.547945
MasVnrType       0.547945
Electrical       0.068493
Id               0.000000
dtype: float64

<hr>
<h1>Vemos que </h1>
... as categorias Fence, Alley, MiscHeature e PoolQC não são informadas em mais de 80% dos casos

Devido ao baixo número de dados disponíveis, escolhi eliminar inteiramente essas colunas! 


In [24]:
# ver o shape antes
print("Antes:\t", df.shape)

df.drop(columns=['Fence','Alley','MiscFeature','PoolQC'],inplace=True)
# ver o shape depois
print("Depois:\t",df.shape)


Antes:	 (1460, 81)
Depois:	 (1460, 77)


<hr>
<h3>Vamos rever a contagem (%) de dados faltantes e tomar mais uma decisão</h3>

In [25]:
df.isnull().sum().sort_values(ascending=False)[0:20]*100/len(df)

FireplaceQu     47.260274
LotFrontage     17.739726
GarageType       5.547945
GarageYrBlt      5.547945
GarageFinish     5.547945
GarageQual       5.547945
GarageCond       5.547945
BsmtExposure     2.602740
BsmtFinType2     2.602740
BsmtQual         2.534247
BsmtCond         2.534247
BsmtFinType1     2.534247
MasVnrArea       0.547945
MasVnrType       0.547945
Electrical       0.068493
KitchenAbvGr     0.000000
BedroomAbvGr     0.000000
HalfBath         0.000000
FullBath         0.000000
BsmtHalfBath     0.000000
dtype: float64

<hr>
Vou apagar as linhas com NAN quando a % de faltantes por coluna for menor que 5%.

Ao tomar essa decisão, assumi a hipótese de que ao remover algo em torno de 5% da amostra, o restante conterá informações suficientes para continuarmos com a análise.


Neste caso, diferentemente do anterior, removo apenas a linha com o dado faltante, e não toda a coluna!




In [26]:
print("Antes:\t", df.shape)
df.dropna(subset=['BsmtExposure','BsmtFinType2','BsmtQual','BsmtCond','BsmtFinType1','MasVnrArea','MasVnrType','Electrical'], inplace=True)

# ver o shape depois
print("Depois:\t",df.shape)



Antes:	 (1460, 77)
Depois:	 (1412, 77)


<hr>
Notemos que houve uma redução de 1460 para 1412 (48 linhas foram removidas, ou cerca de 3%)

Vamos listar novamente a base de dados e ver como ficou:

In [27]:
df.isnull().sum().sort_values(ascending=False)[0:10]*100/len(df)

FireplaceQu     46.529745
LotFrontage     17.776204
GarageFinish     5.240793
GarageYrBlt      5.240793
GarageQual       5.240793
GarageCond       5.240793
GarageType       5.240793
GrLivArea        0.000000
BsmtFullBath     0.000000
BsmtHalfBath     0.000000
dtype: float64

<hr>
<h2> Até aqui, tratamos os dados faltantes com base em duas decisões:</h2>

<li>1 - Remover completamente colunas com mais de 80% de dados faltantes;</li>
<li>2 - Quando a coluna conter menos que 5% de faltantes, remover apenas as linhas comprometidas;</li>
<hr>
<h2> Agora vamos adotar uma estratégia diferente.</h2> <h3>Preencher o restante dos dados faltantes com a mediana da coluna</h3>
Essa estratégia é interessante quando temos um número intermediário de dados faltantes, e removê-los seria potencialmente prejudicial.

Farei a substituição usando o comando .fillna().
Observe que temos parâmetros numéricos e não numéricos. 
No caso numérico, utilizo a mediana, e no caso de dados categóricos, utilizo a classe mais frequênte obtida com .value_counts()[0]:

In [28]:
number2fill=['LotFrontage','GarageYrBlt']
class2fill=['GarageFinish','GarageQual','GarageCond','GarageType','FireplaceQu']

for param in number2fill:
    median = df[param].median()
    df[param].fillna(median, inplace=True)
for param in class2fill:
    class_freq=df[param].value_counts()[0]
    df[param].fillna(class_freq,inplace=True)
    

In [9]:
df.isnull().sum().sort_values(ascending=False)[0:10]*100/len(df)

Id              0.0
HalfBath        0.0
FireplaceQu     0.0
Fireplaces      0.0
Functional      0.0
TotRmsAbvGrd    0.0
KitchenQual     0.0
KitchenAbvGr    0.0
BedroomAbvGr    0.0
FullBath        0.0
dtype: float64

<h1> Pronto! É isso! </h1>

Vou salvar o arquivo para continuarmos a análise em Outliers.ipynb!

In [31]:
df.to_csv('data/train_NoNAN.csv')